In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# Set the location of the Delta Lake and Kafka packages
delta_package = "io.delta:delta-spark_2.12:3.0.0"  # Replace with the correct Delta version
kafka_package = "org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1"  # Replace with the correct Spark version
xml_package = "com.databricks:spark-xml_2.12:0.14.0"

# Initialize Spark Session with Delta Lake and Kafka support
spark = SparkSession.builder \
    .appName("test") \
    .config("spark.jars.packages", f"{delta_package},{xml_package}") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

In [ ]:
from pyspark.sql.functions import col

raw_df = spark.read.format('parquet').option("mergeSchema", "true").load('hdfs://spark-test1:9000/raw/transactions')
raw_df_20 = raw_df.orderBy(col("_raw_insert_timestamp").desc()).limit(20)
print(raw_df.count())

In [ ]:
processed_df = spark.read.format('delta').option("mergeSchema", "true").load('hdfs://spark-test1:9000/processed/transactions')
processed_df_20 = processed_df.orderBy(col("_processed_insert_timestamp").desc()).limit(20)